# Visualizing Data

This notebook is dedicated to visualizing the data to help understand more what it looks like, and setting up the scripts that will clean up the data for 3D Sparse Tensor Network model training.

In [ ]:
#imports
import h5py
import numpy as np
import os.path
import click
import matplotlib.pyplot as plt

## Load Data

The h5 file should be copied from the directory provided in the *data* Slack channel.

In [ ]:
LOADFROM = "../mg22simulated/"
H5 = "output_digi_HDF_Mg22_Ne20pp_8MeV.h5"

file = h5py.File(LOADFROM + H5, 'r')

Format the data into a numpy array. The conversion from h5 to numpy array takes a while.

In [ ]:
file = h5py.File(LOADFROM + H5, 'r')

original_keys = list(file.keys())
original_length = len(original_keys)

event_lens = np.zeros(original_length, int)
for i in range(original_length):
    event = original_keys[i]
    event_lens[i] = len(file[event])
    
ISOTOPE = 'Mg22'
file_name = ISOTOPE + '_w_key_index'
# **only doing this if the file doens't exist already, as the conversion takes a while**
if not os.path.exists(LOADFROM + file_name + '.npy'):
    event_data = np.zeros((original_length, np.max(event_lens), 13), float) 
    for n in range(len(original_keys)):
        name = original_keys[n]
        event = file[name]
        ev_len = len(event)
        #converting event into an array
        for i,e in enumerate(event):
            instant = np.array(list(e))
            event_data[n][i][:12] = np.array(instant)
            event_data[n][i][-1] = float(n) #insert index value to find corresponding event ID
    np.save(LOADFROM + file_name, event_data)

Check the shape of the data. It should be (10000, 1476, 13).

In [ ]:
data = np.load(LOADFROM + ISOTOPE + '_w_key_index' + '.npy')
print(f'Data Shape = {data.shape}') # Expected output: (10000, 1476, 13)

We only want to use the x, y, z, amplitude, and track ID columns, so we will slice out the useful data.

In [ ]:
# slice useful elements
# [0] - x, [1] - y, [2] - z, [4] - amp, [5] - track_id 
sliced_data = data[:, :, [0, 1, 2, 4, 5]]

## Visualizing the Data

We need to understand what are the useful parts of the data, since there are junk events with empty beam. To do that, we want to understand the distribution of the data and visualize it with plots.

First, we look at the distribution of the number of detections for each event. This is a good indicator of empty events, as the number of detections will give us an idea of which events are simply single beam junk events.

From the plot below, we see that a large number of events have less than around 60-70 detections, so it would be reasonable to assume those as junk beam events with no nuclear reaction.

In [ ]:
# Set the figure size
plt.figure(figsize=(12, 12))  # Adjusted height to accommodate both subplots

# First subplot for the unlogged histogram
plt.subplot(2, 1, 1)  # 2 rows, 1 column, 1st plot
plt.hist(event_lens, bins=100, edgecolor='black', alpha=0.5, color='blue')
plt.title("Event Lengths")
plt.xlabel("Detections")
plt.ylabel("Frequency")
xticks = np.arange(0, 1600, 50)
plt.xticks(xticks, rotation=45)

# Second subplot for the logged histogram
plt.subplot(2, 1, 2)  # 2 rows, 1 column, 2nd plot
plt.hist(event_lens, bins=100, edgecolor='black', alpha=0.5, color='red', log=True)
plt.title("Logged Event Lengths")
plt.xlabel("Detections")
plt.ylabel("Log Frequency")
xticks = np.arange(0, 1600, 50)
plt.xticks(xticks, rotation=45)

# Adjust spacing between subplots
plt.tight_layout()

# Show the plots
plt.show()

Next we also want to know around how many tracks there are in an event. We can drop all tracks with track ID 0, since those are just empty detections. We see that there is a maximum of 6 tracks for an event, indicating 6 different types of particles? (What does each track ID even represent?)

We see that track ID 4 has the highest frequency, indicating it is detected the most, while track ID 5, 6 are present, but are rarely detected.

In [ ]:
# Flatten the tracks array and remove zeros
tracks = sliced_data[:, :, -1]
tracksflat = tracks.flatten()
no_zeros = tracksflat[tracksflat != 0]

# Set the figure size
plt.figure(figsize=(6, 6))  # Adjusted height to accommodate both subplots

# First subplot for the unlogged histogram
plt.subplot(2, 1, 1)  # 2 rows, 1 column, 1st plot
plt.hist(no_zeros, bins=6, edgecolor='black', color='blue', alpha=0.5)
plt.title("Track Distribution (Unlogged)")
plt.xlabel("Track ID")
plt.ylabel("Frequency")

# Second subplot for the logged histogram
plt.subplot(2, 1, 2)  # 2 rows, 1 column, 2nd plot
plt.hist(no_zeros, bins=6, edgecolor='black', color='red', alpha=0.5, log=True)
plt.title("Track Distribution (Logged)")
plt.xlabel("Track ID")
plt.ylabel("Log Frequency")

# Adjust spacing between subplots
plt.tight_layout()

# Show the plots
plt.show()

## Cleaning Data

As seen from the histograms above, there is a large number of events with a low number of detections. Those events are likely empty beam events, and so based on the binning of the histogram, events with under 70 detections will not be used for training. We will filter out these events. After the filtering for only events with more than 70 events, we see that there should now be 4591 events left.

In [ ]:
LENEVTS = len(sliced_data) # number of events (10000)
LENDETS = len(sliced_data[0]) # number of detections (1476)
NUMCLASSES = 5 # x, y, z, amp, track_id
cutoff = 70 # discard events with less than 70 detections
newLen = np.sum(event_lens > 70)

new_data = np.zeros((newLen, LENDETS, NUMCLASSES), float)
new_data_index = 0

for i in range(LENEVTS):
    if event_lens[i] > 70:
        new_data[new_data_index] = sliced_data[i] 
        new_data_index += 1

print(f'New Number of Events = {len(new_data)}')

## Visualizing and Plotting 3D Data

The following plots are to visualize what each event and their tracks look like. Each plot is event specific. Change  variable *event* to plot different events in the cell below.

In [ ]:
event = 2 # <--------- select event to plot
selected = new_data[event]
unique_values = np.unique(selected[:, -1])

split_arrays = {}
for value in unique_values:
    split_arrays[value] = selected[selected[:, -1] == value]

We visualize what each track ID looks like in the following 3D plots. Each plot is for a specific track ID, with all the track IDs overlayed with each other in the final plot.

We plot each of the tracks exept for the track ID of 0, as those are just empty events. Each event has 1476 detections total, but most of them are those empty events. We see this as the island of dots with an amplitude of 0 in our final plot.

In [ ]:
# Plotting Each Track
for index, (key, value) in enumerate(split_arrays.items()): 
    if key != 0.0:
        dict_data = split_arrays[key]
        
        x = dict_data[:, 0]
        y = dict_data[:, 1]
        z = dict_data[:, 2]
        amplitude = dict_data[:, 3]
        log_amplitude = np.log(amplitude + 1)
        
        # Subplot setup
        fig = plt.figure(figsize=(15, 10))
        angles = [(30, 0), (30, 45), (30, 90), (30, 135), (30, 180), (30, 225)]
        
        for i, (elev, azim) in enumerate(angles, start=1):
            ax = fig.add_subplot(2, 3, i, projection='3d')
            sc = ax.scatter(x, y, z, c=log_amplitude, cmap='viridis', s=50)

            # Set viewpoint
            ax.view_init(elev, azim)
            
            ax.set_xlabel('X Label')
            ax.set_ylabel('Y Label')
            ax.set_zlabel('Z Label')
            ax.set_title(f"View: elevation={elev}, azimuth={azim}")
        
        # Colorbar; since we have multiple subplots, we adjust the position for better layout
        cax = fig.add_axes([0.92, 0.3, 0.02, 0.4]) 
        cbar = fig.colorbar(sc, cax=cax)
        cbar.set_label('Log Amplitude')
        
        fig.suptitle(f"3D Scatter Plot of Track {int(key)}, Event {event}, ")
        plt.show()

# Plot all tracks overlayed with each other  
x = selected[:, 0]
y = selected[:, 1]
z = selected[:, 2]
amplitude = selected[:, 3]
log_amplitude = np.log(amplitude + 1)

# Subplot setup
fig = plt.figure(figsize=(15, 10))
angles = [(30, 0), (30, 45), (30, 90), (30, 135), (30, 180), (30, 225)]

for i, (elev, azim) in enumerate(angles, start=1):
    ax = fig.add_subplot(2, 3, i, projection='3d')
    sc = ax.scatter(x, y, z, c=log_amplitude, cmap='viridis', s=50)

    
    # Set viewpoint
    ax.view_init(elev, azim)
    
    ax.set_xlabel('X Label')
    ax.set_ylabel('Y Label')
    ax.set_zlabel('Z Label')
    ax.set_title(f"View: elevation={elev}, azimuth={azim}")

# Colorbar; since we have multiple subplots, we adjust the position for better layout
cax = fig.add_axes([0.92, 0.3, 0.02, 0.4]) 
cbar = fig.colorbar(sc, cax=cax)
cbar.set_label('Log Amplitude')

fig.suptitle(f"3D Scatter Plot of All Tracks, Event {event}")
plt.show()

## Removing the Zero Events

We currently have saved the cleaned data as numpy arrays, and they will be fed into a train-validation-test split script. But we need to get rid of the many empty zero events so that the model doesn't train on empty detections. The code below will a pipeline illustrating a small scale model to be used for testing and understanding what we are doing to the data using PyTorch's dataloader.

We first load in the train test split data.

In [ ]:
import tqdm
import torch
import torchsparse
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torch.cuda import amp
from torchsparse import SparseTensor, nn as spnn
from torchsparse.utils.collate import sparse_collate
from sklearn.metrics import confusion_matrix

import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from sklearn.preprocessing import StandardScaler

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, coords, feats, labels):
        self.coords = coords
        self.feats = feats
        self.labels = labels
        
    def __len__(self):
        return len(self.coords)

    def __getitem__(self, idx):
        return self.coords[idx], self.feats[idx], self.labels[idx]

In [ ]:
loadfrom = "../mg22simulated/"
ISOTOPE = "Mg22"

coords_train = np.load(loadfrom + ISOTOPE + "_coords_train.npy")
coords_val = np.load(loadfrom + ISOTOPE + "_coords_val.npy")
feats_train = np.load(loadfrom + ISOTOPE + "_feats_train.npy")
feats_val = np.load(loadfrom + ISOTOPE + "_feats_val.npy")
labels_train = np.load(loadfrom + ISOTOPE + "_labels_train.npy")
labels_val = np.load(loadfrom + ISOTOPE + "_labels_val.npy")

We only use 100 of the events for our small scale model training and 50 events for validation.

In [ ]:
coords_train = coords_train[0:100]
feats_train = feats_train[0:100]
labels_train = labels_train[0:100]
    
coords_val = coords_val[0:50]
feats_val = feats_val[0:50]
labels_val = labels_val[0:50]

Below we create the layers and define the criterion and opitmizer for our model.

In [ ]:
# GPU Settings
device = 'cuda'
amp_enabled = True

model = nn.Sequential(
    spnn.Conv3d(4, 32, 3),
    spnn.BatchNorm(32),
    spnn.ReLU(True),
    spnn.Conv3d(32, 32, 3),
    spnn.BatchNorm(32),
    spnn.ReLU(True),
    spnn.Conv3d(32, 7, 1),
).to(device)

lr = 1e-3
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scaler = amp.GradScaler(enabled=amp_enabled)

We create our dataloaders from the training and validation data sets

In [ ]:
num_epochs = 100
batch_size = 12

train_set = CustomDataset(coords_train, feats_train, labels_train)
train_loader = DataLoader(train_set, batch_size=batch_size)

val_set = CustomDataset(coords_val, feats_val, labels_val)
val_loader = DataLoader(val_set, batch_size=batch_size)

train_steps = len(train_loader)

Training Loop

In [ ]:
training_losses = []
validation_losses = []

for epoch in range(num_epochs):

    model.train()
    running_loss = 0.0
    
    for batch_idx, (batch_coords, batch_feats, batch_labels) in enumerate(train_loader):
        tr_inputs_list = []
        tr_labels_list = []
        
        for i in range(len(batch_coords)):
            current_coords = batch_coords[i]
            current_feats = batch_feats[i]
            current_labels = batch_labels[i]

            mask = ~(current_coords == 0).all(axis=1)

            # Apply the mask to the array
            current_coords = current_coords[mask]
            current_feats = current_feats[mask]
            current_labels = current_labels[mask]
            
            current_coords = torch.tensor(current_coords, dtype=torch.int)
            current_feats = torch.tensor(current_feats, dtype=torch.float)
            current_labels = torch.tensor(current_labels, dtype=torch.long)

            inputs_sparse = SparseTensor(coords=current_coords, feats=current_feats)
            labels_sparse = SparseTensor(coords=current_coords, feats=current_labels)
            tr_inputs_list.append(inputs_sparse)
            tr_labels_list.append(labels_sparse)
        
        tr_inputs = sparse_collate(tr_inputs_list).to(device=device)
        tr_labels = sparse_collate(tr_labels_list).to(device=device)
        
        with amp.autocast(enabled=amp_enabled):
            tr_outputs = model(tr_inputs)
            tr_labelsloss = tr_labels.feats.squeeze(-1)
            tr_loss = criterion(tr_outputs.feats, tr_labelsloss)
        
        running_loss += tr_loss.item()
    
        optimizer.zero_grad()
        scaler.scale(tr_loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
    training_losses.append(running_loss / train_steps)
    print(f"[Epoch {epoch+1}] Running Loss: {running_loss / train_steps}")

    model.eval()
    torchsparse.backends.benchmark = True  # type: ignore
    val_loss = 0.0

In [ ]:
coords_test = np.load(loadfrom + ISOTOPE + "_coords_test.npy")
feats_test = np.load(loadfrom + ISOTOPE + "_feats_test.npy")
labels_test = np.load(loadfrom + ISOTOPE + "_labels_test.npy")

runninglen = 0

test_set = CustomDataset(coords_test, feats_test, labels_test)
test_loader = DataLoader(test_set, batch_size=batch_size)

model.load_state_dict(torch.load('../training/2023-11-10-19:25:27/models/epochs100_lr0.001_2023-11-10-19:25:27.pth'))
model.eval()

torchsparse.backends.benchmark = True  # type: ignore

with torch.no_grad():
    
    all_preds = []
    all_labels = []
    all_preds = np.array(all_preds)
    
    total_correct = 0
    for batch_idx, (batch_coords, batch_feats, batch_labels) in enumerate(test_loader):
        t_inputs_list = []
        t_labels_list = []

        for i in range(len(batch_coords)):
            current_coords = batch_coords[i]
            current_feats = batch_feats[i]
            current_labels = batch_labels[i]
            
            mask = ~(current_coords == 0).all(axis=1)

            # Apply the mask to the array
            current_coords = current_coords[mask]
            current_feats = current_feats[mask]
            current_labels = current_labels[mask]
            all_labels = np.concatenate((all_labels, current_labels.reshape(-1)))
            
            current_coords = torch.tensor(current_coords, dtype=torch.int)
            current_feats = torch.tensor(current_feats, dtype=torch.float)
            current_labels = torch.tensor(current_labels, dtype=torch.long)
            
            t_inputs_sparse = SparseTensor(coords=current_coords, feats=current_feats)
            t_labels_sparse = SparseTensor(coords=current_coords, feats=current_labels)
            t_inputs_list.append(t_inputs_sparse)
            t_labels_list.append(t_labels_sparse)

            runninglen += len(current_coords)
        t_inputs = sparse_collate(t_inputs_list).to(device=device)
        t_labels = sparse_collate(t_labels_list).to(device=device)

        
        n_correct = 0
        
        with amp.autocast(enabled=True):
            outputs = model(t_inputs)
            
            labelsloss = t_labels.feats.squeeze(-1)
            loss = criterion(outputs.feats, labelsloss)
            _, predicted = torch.max(outputs.feats, 1)

            
            all_preds = np.concatenate((all_preds, predicted.cpu().numpy()))
            n_correct += (predicted == labelsloss).sum().item()
            total_correct += n_correct
    
    acc = 100.0 * total_correct / (len(all_preds))
    print(f'Accuracy of the model: {acc:.3g} %')

## Confusion Matrices

We generate confusion matrices to visualize how our model classifies each respective particle track by comparing True Labels vs Predicted Labels.

In [ ]:
cm = confusion_matrix(all_labels.astype(int), all_preds.astype(int), labels=[1, 2, 3, 4, 5, 6])

# Normalize by row
plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
plt.title("Confusion Matrix")
plt.colorbar()

tick_marks = np.arange(len([1, 2, 3, 4, 5, 6]))  # Assuming labels are [1, 2, 3, 4, 5, 6]
plt.xticks(tick_marks, [1, 2, 3, 4, 5, 6])
plt.yticks(tick_marks, [1, 2, 3, 4, 5, 6])
plt.ylabel('True label')
plt.xlabel('Predicted label')

thresh = cm.max() / 2
for i, j in np.ndindex(cm.shape):
    plt.text(j, i, f'{cm[i, j]:}',
             horizontalalignment="center",
             color="white" if cm[i, j] > thresh else "black")

plt.show()

We can also normalize the matrices by rows or columns depending on what we want to learn from the confusion matrix

In [ ]:
cm = confusion_matrix(all_labels.astype(int), all_preds.astype(int), labels=[1, 2, 3, 4, 5, 6])

# Normalize by rows
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

plt.imshow(cm_normalized, interpolation='nearest', cmap=plt.cm.Blues)
plt.title("Normalized Confusion Matrix")
plt.colorbar()

tick_marks = np.arange(len([1, 2, 3, 4, 5, 6]))
plt.xticks(tick_marks, [1, 2, 3, 4, 5, 6])
plt.yticks(tick_marks, [1, 2, 3, 4, 5, 6])
plt.ylabel('True label')
plt.xlabel('Predicted label')

thresh = cm_normalized.max() / 2
for i, j in np.ndindex(cm_normalized.shape):
    plt.text(j, i, f'{cm_normalized[i, j]:.2f}',
             horizontalalignment="center",
             color="white" if cm_normalized[i, j] > thresh else "black")

plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix
import numpy as np
import matplotlib.pyplot as plt

cm = confusion_matrix(all_labels.astype(int), all_preds.astype(int), labels=[1, 2, 3, 4, 5, 6])

# Normalize by columns
column_sums = cm.sum(axis=0)
# Avoid division by zero
column_sums[column_sums == 0] = 1
cm_normalized_col = cm.astype('float') / column_sums[np.newaxis, :]

plt.imshow(cm_normalized_col, interpolation='nearest', cmap=plt.cm.Blues)
plt.title("Column-Normalized Confusion Matrix")
plt.colorbar()

tick_marks = np.arange(len([1, 2, 3, 4, 5, 6]))
plt.xticks(tick_marks, [1, 2, 3, 4, 5, 6])
plt.yticks(tick_marks, [1, 2, 3, 4, 5, 6])
plt.ylabel('True label')
plt.xlabel('Predicted label')

thresh = cm_normalized_col.max() / 2
for i, j in np.ndindex(cm_normalized_col.shape):
    plt.text(j, i, f'{cm_normalized_col[i, j]:.2f}',
             horizontalalignment="center",
             color="white" if cm_normalized_col[i, j] > thresh else "black")

plt.show()
